In [123]:
import pandas as pd

In [124]:
df = pd.read_csv("data_unified.csv")

In [125]:
# Carregar dados filtrados
import numpy as np
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

df_filtered = pd.read_csv("data_unified_filtered.csv", decimal=',', encoding='utf-8')

# Converter colunas numéricas (usando vírgula como decimal)
numeric_cols = ['volume_projetado', 'elasticidade', 'base_preco_bruto_unit', 
                'base_preco_liquido_unit', 'base_gvv_labor_unit', 
                'base_margem_variavel_unit', 'capacidade_min', 'capacidade_max']

for col in numeric_cols:
    if col in df_filtered.columns:
        df_filtered[col] = df_filtered[col].astype(str).str.replace(',', '.').astype(float)

print(f"Dados carregados: {len(df_filtered)} linhas")
print(f"SKUs únicos: {df_filtered['chave_sku'].nunique()}")
print(f"Grupos de capacidade: {df_filtered['grupo_capacidade'].nunique()}")
print(f"\nGrupos de capacidade únicos:")
print(df_filtered['grupo_capacidade'].dropna().unique())

Dados carregados: 695 linhas
SKUs únicos: 35
Grupos de capacidade: 10

Grupos de capacidade únicos:
['Mini Lata|220ml' 'Vidro não Retornável|250ml' 'KS|290-310ml' 'Pet|200ml'
 'Lata|350ml' 'Pet|600ml' 'Pet|1-1.5L' 'LS|1L' 'BIB|5-18L' 'Pet|2-3L']


In [126]:
# Agregar dados por TIPO dentro de cada GRUPO_CAPACIDADE
# IMPORTANTE: A capacidade é compartilhada por grupo, mas as demandas são por TIPO
# Ex: Pet|1.0 e Pet|1.5 compartilham capacidade Pet|1-1.5L
# Devemos otimizar a distribuição da capacidade entre os TIPOs, não entre SKUs individuais

# Filtrar por mês se necessário
if 'depara_mess' in df_filtered.columns:
    meses_unicos = df_filtered['depara_mess'].dropna().unique()
    if len(meses_unicos) > 1:
        mes_selecionado = sorted(meses_unicos)[0]
        print(f"⚠ Múltiplos meses encontrados. Usando apenas: {mes_selecionado}")
        df_filtered = df_filtered[df_filtered['depara_mess'] == mes_selecionado].copy()

# Agregar por TIPO dentro de cada GRUPO_CAPACIDADE
# Isso agrupa Pet|1.0 e Pet|1.5 separadamente, mas ambos compartilham a mesma capacidade
df_work = df_filtered.groupby(['grupo_capacidade', 'tipo']).agg({
    'volume_projetado': 'sum',  # Soma demanda de todos os SKUs do mesmo tipo
    'elasticidade': 'mean',
    'base_margem_variavel_unit': 'mean',  # Média da margem unitária do tipo
    'base_preco_liquido_unit': 'mean',
    'capacidade_min': 'first',  # Capacidade é por grupo, não por tipo
    'capacidade_max': 'first',
    'brand': lambda x: ', '.join(x.unique()[:3]),  # Lista de marcas (para referência)
    'package': 'first',
    'returnability': 'first'
}).reset_index()

print(f"\nDados agregados por TIPO dentro de GRUPO_CAPACIDADE: {len(df_work)} tipos únicos")
print(f"Volume total projetado: {df_work['volume_projetado'].sum():,.0f} UC")
print(f"Grupos de capacidade: {df_work['grupo_capacidade'].nunique()}")
print(f"Tipos com capacidade definida: {df_work['capacidade_max'].notna().sum()}")

# Mostrar exemplo
print(f"\nExemplo de agregação:")
exemplo = df_work[df_work['grupo_capacidade'] == 'Pet|1-1.5L'][['tipo', 'volume_projetado', 'capacidade_max']]
print(exemplo.to_string(index=False))

⚠ Múltiplos meses encontrados. Usando apenas: 2025-06-01

Dados agregados por TIPO dentro de GRUPO_CAPACIDADE: 15 tipos únicos
Volume total projetado: 108,491,656 UC
Grupos de capacidade: 10
Tipos com capacidade definida: 15

Exemplo de agregação:
   tipo  volume_projetado  capacidade_max
Pet|1.0      2.973519e+06    6.314176e+06
Pet|1.5      5.999084e+06    6.314176e+06


In [127]:
def optimize_single_tipo_group(df_grupo, grupo, cap_min, cap_max, demandas, lucros_unit, indices_tipo):
    """
    Otimiza grupos com APENAS 1 TIPO.
    Modelo mais simples: apenas verificar capacidade e demanda.
    """
    n_tipos = len(df_grupo)
    if n_tipos != 1:
        return None
    
    demanda = demandas[0]
    lucro_unit = lucros_unit[0]
    idx_tipo = indices_tipo[0]
    
    # Para 1 TIPO: simplesmente usar o mínimo entre demanda e capacidade máxima
    # E garantir que seja >= capacidade mínima
    volume_otimizado = min(demanda, cap_max) if cap_max < float('inf') else demanda
    
    # Se volume < cap_min e demanda >= cap_min, usar cap_min
    if cap_min > 0 and volume_otimizado < cap_min and demanda >= cap_min:
        volume_otimizado = min(cap_min, cap_max) if cap_max < float('inf') else cap_min
    
    # Não pode exceder demanda
    volume_otimizado = min(volume_otimizado, demanda)
    
    lucro_total = volume_otimizado * lucro_unit
    atendimento_pct = (volume_otimizado / demanda * 100) if demanda > 0 else 0
    
    # Status
    if volume_otimizado > cap_max and cap_max < float('inf'):
        status_grupo = 'Acima Máximo'
    elif volume_otimizado < cap_min and cap_min > 0:
        status_grupo = 'Abaixo Mínimo'
    else:
        status_grupo = 'OK'
    
    return {
        'volumes_otimizados': np.array([volume_otimizado]),
        'lucro_total': lucro_total,
        'atendimento_pct': atendimento_pct,
        'status_grupo': status_grupo,
        'volume_total_final': volume_otimizado
    }

def optimize_multi_tipo_group(df_grupo, grupo, cap_min, cap_max, demandas, lucros_unit, indices_tipo):
    """
    Otimiza grupos com MÚLTIPLOS TIPOs (2+).
    Usa otimização SLSQP para distribuir capacidade entre TIPOs.
    """
    n_tipos = len(df_grupo)
    if n_tipos < 2:
        return None
    
    demanda_total = demandas.sum()
    
    # Função objetivo: maximizar lucro total (minimizar negativo)
    def objetivo(x):
        lucro_total = -np.sum(lucros_unit * x)
        
        # Penalidades
        volume_total = np.sum(x)
        
        # Penalidade por exceder capacidade máxima
        if volume_total > cap_max and cap_max < float('inf'):
            lucro_total += 1e10 * (volume_total - cap_max)
        
        # Penalidade por não atingir capacidade mínima
        if volume_total < cap_min and cap_min > 0 and cap_min < cap_max:
            lucro_total += 1e6 * (cap_min - volume_total)
        
        # Penalidade por exceder demanda individual
        excesso_demanda = np.sum(np.maximum(0, x - demandas))
        if excesso_demanda > 0:
            lucro_total += 1e8 * excesso_demanda
        
        # Penalidade por volumes negativos
        volumes_negativos = np.sum(np.maximum(0, -x))
        if volumes_negativos > 0:
            lucro_total += 1e10 * volumes_negativos
        
        return lucro_total
    
    # Restrições
    constraints = []
    
    # Restrição: volume total <= capacidade máxima
    if cap_max < float('inf'):
        constraints.append({
            'type': 'ineq',
            'fun': lambda x: cap_max - np.sum(x)
        })
    
    # Bounds: apenas limitar pela demanda individual
    bounds = [(0.0, dem) for dem in demandas]
    
    # Restrição: volume total >= capacidade mínima (se viável)
    soma_bounds_max = sum(b[1] for b in bounds)
    if (cap_min > 0 and 
        cap_min < cap_max and
        soma_bounds_max >= cap_min):
        constraints.append({
            'type': 'ineq',
            'fun': lambda x: np.sum(x) - cap_min
        })
    
    # Ponto inicial: priorizar TIPOs mais lucrativos
    x0 = np.zeros(n_tipos)
    idxs_ordenados = np.argsort(lucros_unit)[::-1]
    
    if cap_max < float('inf') and demanda_total > cap_max:
        # Demanda excede capacidade: alocar para os mais lucrativos
        capacidade_restante = cap_max
        for idx in idxs_ordenados:
            if capacidade_restante <= 0:
                break
            alocacao = min(demandas[idx], capacidade_restante)
            x0[idx] = alocacao
            capacidade_restante -= alocacao
    else:
        # Se cabe tudo, usar demanda
        x0 = demandas.copy()
    
    # Verificar viabilidade
    soma_bounds_max = sum(b[1] for b in bounds)
    viável = True
    
    if cap_max < float('inf'):
        if cap_min > 0 and soma_bounds_max < cap_min:
            viável = False
        elif cap_min > cap_max:
            viável = False
    
    # Otimizar
    try:
        if viável:
            result = minimize(
                objetivo,
                x0,
                method='SLSQP',
                bounds=bounds,
                constraints=constraints,
                options={'maxiter': 1000, 'ftol': 1e-6, 'disp': False}
            )
        else:
            result = type('obj', (object,), {'success': False, 'message': 'Problema inviável', 'x': x0})()
        
        if result.success:
            volumes_otimizados = np.maximum(0, result.x)
            volumes_otimizados = np.minimum(volumes_otimizados, demandas)
            
            # Ajustar se exceder capacidade total
            volume_total = volumes_otimizados.sum()
            if volume_total > cap_max and cap_max < float('inf'):
                # Redistribuir capacidade para os mais lucrativos
                idxs_ordenados = np.argsort(lucros_unit)[::-1]
                volumes_otimizados = np.zeros(n_tipos)
                capacidade_restante = cap_max
                for idx in idxs_ordenados:
                    if capacidade_restante <= 0:
                        break
                    alocacao = min(demandas[idx], capacidade_restante)
                    volumes_otimizados[idx] = alocacao
                    capacidade_restante -= alocacao
            
            volume_total_final = volumes_otimizados.sum()
            lucro_total = np.sum(lucros_unit * volumes_otimizados)
            atendimento_pct = (volume_total_final / demanda_total * 100) if demanda_total > 0 else 0
            
            # Status
            if volume_total_final > cap_max and cap_max < float('inf'):
                status_grupo = 'Acima Máximo'
            elif volume_total_final < cap_min and cap_min > 0:
                status_grupo = 'Abaixo Mínimo'
            else:
                status_grupo = 'OK'
            
            return {
                'volumes_otimizados': volumes_otimizados,
                'lucro_total': lucro_total,
                'atendimento_pct': atendimento_pct,
                'status_grupo': status_grupo,
                'volume_total_final': volume_total_final
            }
        else:
            # Fallback: alocar por lucratividade
            idxs_ordenados = np.argsort(lucros_unit)[::-1]
            volumes_fallback = np.zeros(n_tipos)
            capacidade_restante = cap_max if cap_max < float('inf') else demanda_total
            
            for idx in idxs_ordenados:
                if capacidade_restante <= 0:
                    break
                alocacao = min(demandas[idx], capacidade_restante)
                volumes_fallback[idx] = alocacao
                capacidade_restante -= alocacao
            
            volume_total_final = volumes_fallback.sum()
            lucro_total = np.sum(lucros_unit * volumes_fallback)
            atendimento_pct = (volume_total_final / demanda_total * 100) if demanda_total > 0 else 0
            
            status_grupo = 'OK'
            if volume_total_final > cap_max and cap_max < float('inf'):
                status_grupo = 'Acima Máximo'
            elif volume_total_final < cap_min and cap_min > 0:
                status_grupo = 'Abaixo Mínimo'
            
            return {
                'volumes_otimizados': volumes_fallback,
                'lucro_total': lucro_total,
                'atendimento_pct': atendimento_pct,
                'status_grupo': status_grupo,
                'volume_total_final': volume_total_final
            }
    except Exception as e:
        # Fallback em caso de erro
        idxs_ordenados = np.argsort(lucros_unit)[::-1]
        volumes_fallback = np.zeros(n_tipos)
        capacidade_restante = cap_max if cap_max < float('inf') else demanda_total
        
        for idx in idxs_ordenados:
            if capacidade_restante <= 0:
                break
            alocacao = min(demandas[idx], capacidade_restante)
            volumes_fallback[idx] = alocacao
            capacidade_restante -= alocacao
        
        volume_total_final = volumes_fallback.sum()
        lucro_total = np.sum(lucros_unit * volumes_fallback)
        atendimento_pct = (volume_total_final / demanda_total * 100) if demanda_total > 0 else 0
        
        return {
            'volumes_otimizados': volumes_fallback,
            'lucro_total': lucro_total,
            'atendimento_pct': atendimento_pct,
            'status_grupo': 'OK',
            'volume_total_final': volume_total_final
        }

def optimize_by_capacity_group(df_work):
    """
    Otimiza mix de produção por GRUPO DE CAPACIDADE.
    
    Objetivo: Maximizar lucro total
    Restrições:
    - Volume total do grupo <= capacidade_max
    - Volume total do grupo >= capacidade_min (se aplicável)
    - Volume de cada SKU <= demanda (volume_projetado)
    - Volume de cada SKU >= 0
    """
    print("=" * 80)
    print("OTIMIZAÇÃO POR GRUPO DE CAPACIDADE")
    print("=" * 80)
    
    # Preparar resultado
    df_result = df_work.copy()
    df_result['volume_otimizado'] = 0.0
    df_result['lucro_otimizado'] = 0.0
    df_result['atendimento_pct'] = 0.0
    df_result['status_capacidade'] = 'OK'
    
    # Agrupar por grupo_capacidade
    grupos_unicos = df_work['grupo_capacidade'].dropna().unique()
    print(f"\n{len(grupos_unicos)} grupos de capacidade únicos encontrados")
    
    resultados = {}
    
    for grupo in grupos_unicos:
        df_grupo = df_work[df_work['grupo_capacidade'] == grupo].copy()
        
        if df_grupo.empty:
            continue
        
        n_tipos = len(df_grupo)
        print(f"\n{'='*60}")
        print(f"GRUPO: {grupo} ({n_tipos} TIPOs)")
        
        # Mostrar tipos no grupo
        tipos_no_grupo = df_grupo['tipo'].unique()
        print(f"  TIPOs: {', '.join(tipos_no_grupo)}")
        
        # Dados do grupo (agora por TIPO, não por SKU)
        demandas = df_grupo['volume_projetado'].fillna(0).values
        lucros_unit = df_grupo['base_margem_variavel_unit'].fillna(0).values
        indices_tipo = df_grupo.index.tolist()
        
        # Capacidade compartilhada do grupo
        cap_min = df_grupo['capacidade_min'].iloc[0] if df_grupo['capacidade_min'].notna().any() else 0
        cap_max = df_grupo['capacidade_max'].iloc[0] if df_grupo['capacidade_max'].notna().any() else float('inf')
        
        demanda_total = demandas.sum()
        lucro_medio = lucros_unit.mean() if len(lucros_unit) > 0 else 0
        
        print(f"  Demanda total: {demanda_total:,.0f} UC")
        print(f"  Capacidade: {cap_min:,.0f} - {cap_max:,.0f} UC" if cap_max < float('inf') else f"  Capacidade: {cap_min:,.0f} - Inf UC")
        print(f"  Lucro médio: R$ {lucro_medio:.2f}/UC")
        
        # ESCOLHER MODELO BASEADO NO NÚMERO DE TIPOs
        if n_tipos == 1:
            print(f"  📌 Usando modelo SIMPLES (1 TIPO)")
            resultado = optimize_single_tipo_group(
                df_grupo, grupo, cap_min, cap_max, demandas, lucros_unit, indices_tipo
            )
        else:
            print(f"  📌 Usando modelo MULTI-TIPO ({n_tipos} TIPOs)")
            resultado = optimize_multi_tipo_group(
                df_grupo, grupo, cap_min, cap_max, demandas, lucros_unit, indices_tipo
            )
        
        if resultado is None:
            print(f"  ✗ Erro: Não foi possível otimizar")
            continue
        
        # Extrair resultados
        volumes_otimizados = resultado['volumes_otimizados']
        lucro_total = resultado['lucro_total']
        atendimento_pct = resultado['atendimento_pct']
        status_grupo = resultado['status_grupo']
        volume_total_final = resultado['volume_total_final']
        
        # Atualizar resultado POR TIPO (não por grupo)
        for i, idx in enumerate(indices_tipo):
            df_result.at[idx, 'volume_otimizado'] = volumes_otimizados[i]
            df_result.at[idx, 'lucro_otimizado'] = lucros_unit[i] * volumes_otimizados[i]
            df_result.at[idx, 'atendimento_pct'] = (volumes_otimizados[i] / demandas[i] * 100) if demandas[i] > 0 else 0
            df_result.at[idx, 'status_capacidade'] = status_grupo
        
        resultados[grupo] = {
            'volume_total': volume_total_final,
            'demanda_total': demanda_total,
            'lucro_total': lucro_total,
            'atendimento_pct': atendimento_pct,
            'status': status_grupo,
            'n_tipos': n_tipos
        }
        
        print(f"  ✓ Otimizado: {volume_total_final:,.0f} UC")
        print(f"  ✓ Lucro: R$ {lucro_total:,.2f}")
        print(f"  ✓ Atendimento: {atendimento_pct:.1f}%")
        print(f"  ✓ Status: {status_grupo}")
        
        # Mostrar detalhamento por TIPO
        print(f"  📊 Detalhamento por TIPO:")
        for i, idx in enumerate(indices_tipo):
            tipo_nome = df_grupo.loc[idx, 'tipo']
            print(f"    - {tipo_nome}: {volumes_otimizados[i]:,.0f} UC "
                  f"(demanda: {demandas[i]:,.0f}, lucro: R$ {lucros_unit[i] * volumes_otimizados[i]:,.2f})")
        
        # Fim do processamento deste grupo - continuar para o próximo
    
    # Retornar resultados finais
    return df_result, resultados

# Executar otimização
df_result, resultados = optimize_by_capacity_group(df_work)

OTIMIZAÇÃO POR GRUPO DE CAPACIDADE

10 grupos de capacidade únicos encontrados

GRUPO: BIB|5-18L (3 TIPOs)
  TIPOs: BIB|10.0, BIB|18.0, BIB|5.0
  Demanda total: 2,711,957 UC
  Capacidade: 2,255,894 - 3,963,991 UC
  Lucro médio: R$ 0.48/UC
  📌 Usando modelo MULTI-TIPO (3 TIPOs)
  ✓ Otimizado: 2,711,957 UC
  ✓ Lucro: R$ 1,060,405.86
  ✓ Atendimento: 100.0%
  ✓ Status: OK
  📊 Detalhamento por TIPO:
    - BIB|10.0: 809,986 UC (demanda: 809,986, lucro: R$ 509,935.66)
    - BIB|18.0: 1,249,530 UC (demanda: 1,249,530, lucro: R$ 60,040.59)
    - BIB|5.0: 652,442 UC (demanda: 652,442, lucro: R$ 490,429.61)

GRUPO: KS|290-310ml (1 TIPOs)
  TIPOs: Vidro não Retornável|0.29
  Demanda total: 2,135,721 UC
  Capacidade: 805,829 - 1,926,217 UC
  Lucro médio: R$ 0.86/UC
  📌 Usando modelo SIMPLES (1 TIPO)
  ✓ Otimizado: 1,926,217 UC
  ✓ Lucro: R$ 1,655,255.33
  ✓ Atendimento: 90.2%
  ✓ Status: OK
  📊 Detalhamento por TIPO:
    - Vidro não Retornável|0.29: 1,926,217 UC (demanda: 2,135,721, lucro: R$ 1,65

In [128]:
# Calcular volume otimizado total por TIPO
print("\n" + "=" * 80)
print("VOLUME OTIMIZADO TOTAL POR TIPO")
print("=" * 80)

df_tipo_summary = df_result.groupby('tipo').agg({
    'volume_projetado': 'sum',  # Demanda total do tipo
    'volume_otimizado': 'sum',  # Volume otimizado total do tipo
    'lucro_otimizado': 'sum',   # Lucro total otimizado do tipo
    'grupo_capacidade': 'first',
    'capacidade_max': 'first'
}).reset_index()

df_tipo_summary['atendimento_pct'] = (
    (df_tipo_summary['volume_otimizado'] / df_tipo_summary['volume_projetado'] * 100)
    .round(2)
)

df_tipo_summary = df_tipo_summary.sort_values('volume_otimizado', ascending=False)

print(f"\nTotal de TIPOs: {len(df_tipo_summary)}")
print(f"Volume otimizado total: {df_tipo_summary['volume_otimizado'].sum():,.0f} UC")
print(f"Lucro otimizado total: R$ {df_tipo_summary['lucro_otimizado'].sum():,.2f}")
print(f"\n{'TIPO':<30} {'GRUPO_CAPACIDADE':<25} {'DEMANDA':>15} {'OTIMIZADO':>15} {'ATEND.%':>10} {'LUCRO':>15}")
print("-" * 110)

for _, row in df_tipo_summary.iterrows():
    print(f"{row['tipo']:<30} {str(row['grupo_capacidade']):<25} "
          f"{row['volume_projetado']:>15,.0f} {row['volume_otimizado']:>15,.0f} "
          f"{row['atendimento_pct']:>9.1f}% {row['lucro_otimizado']:>15,.2f}")

# Mostrar também em DataFrame
print("\n" + "=" * 80)
print("DataFrame resumido por TIPO:")
print("=" * 80)
display(df_tipo_summary)


VOLUME OTIMIZADO TOTAL POR TIPO

Total de TIPOs: 15
Volume otimizado total: 58,890,804 UC
Lucro otimizado total: R$ 35,847,054.79

TIPO                           GRUPO_CAPACIDADE                  DEMANDA       OTIMIZADO    ATEND.%           LUCRO
--------------------------------------------------------------------------------------------------------------
Pet|2.0                        Pet|2-3L                       49,738,817      25,563,146      51.4%   10,474,559.62
Lata|0.35                      Lata|350ml                     18,122,998      10,668,886      58.9%    9,098,238.99
Pet|0.6                        Pet|600ml                       7,530,622       4,094,682      54.4%    5,082,343.24
Pet|1.5                        Pet|1-1.5L                      5,999,084       3,340,657      55.7%    2,040,912.74
Pet|0.2                        Pet|200ml                       7,732,485       3,324,111      43.0%    1,781,976.88
Pet|1.0                        Pet|1-1.5L                    

,tipo,volume_projetado,volume_otimizado,lucro_otimizado,grupo_capacidade,capacidade_max,atendimento_pct
10,Pet|2.0,4.973882e+07,2.556315e+07,1.047456e+07,Pet|2-3L,2.556315e+07,51.39
5,Lata|0.35,1.812300e+07,1.066889e+07,9.098239e+06,Lata|350ml,1.066889e+07,58.87
7,Pet|0.6,7.530622e+06,4.094682e+06,5.082343e+06,Pet|600ml,4.094682e+06,54.37
9,Pet|1.5,5.999084e+06,3.340657e+06,2.040913e+06,Pet|1-1.5L,6.314176e+06,55.69
6,Pet|0.2,7.732485e+06,3.324111e+06,1.781977e+06,Pet|200ml,3.324111e+06,42.99
8,Pet|1.0,2.973519e+06,2.973519e+06,2.425796e+06,Pet|1-1.5L,6.314176e+06,100.00
14,Vidro não Retornável|1.0,8.174734e+06,2.401492e+06,7.543044e+05,LS|1L,2.401492e+06,29.38
13,Vidro não Retornável|0.29,2.135721e+06,1.926217e+06,1.655255e+06,KS|290-310ml,1.926217e+06,90.19
3,Lata|0.22,2.322536e+06,1.500922e+06,1.142744e+06,Mini Lata|220ml,1.500922e+06,64.62
1,BIB|18.0,1.249530e+06,1.249530e+06,6.004059e+04,BIB|5-18L,3.963991e+06,100.00


In [129]:
# Calcular volume otimizado total por TIPO
print("\n" + "=" * 80)
print("VOLUME OTIMIZADO TOTAL POR TIPO")
print("=" * 80)

df_tipo_summary = df_result.groupby('tipo').agg({
    'volume_projetado': 'sum',  # Demanda total do tipo
    'volume_otimizado': 'sum',  # Volume otimizado total do tipo
    'lucro_otimizado': 'sum',   # Lucro total otimizado do tipo
    'grupo_capacidade': 'first',
    'capacidade_max': 'first'
}).reset_index()

df_tipo_summary['atendimento_pct'] = (
    (df_tipo_summary['volume_otimizado'] / df_tipo_summary['volume_projetado'] * 100)
    .round(2)
)

df_tipo_summary = df_tipo_summary.sort_values('volume_otimizado', ascending=False)

print(f"\nTotal de TIPOs: {len(df_tipo_summary)}")
print(f"Volume otimizado total: {df_tipo_summary['volume_otimizado'].sum():,.0f} UC")
print(f"Lucro otimizado total: R$ {df_tipo_summary['lucro_otimizado'].sum():,.2f}")
print(f"\n{'TIPO':<30} {'GRUPO_CAPACIDADE':<25} {'DEMANDA':>15} {'OTIMIZADO':>15} {'ATEND.%':>10} {'LUCRO':>15}")
print("-" * 110)

for _, row in df_tipo_summary.iterrows():
    print(f"{row['tipo']:<30} {str(row['grupo_capacidade']):<25} "
          f"{row['volume_projetado']:>15,.0f} {row['volume_otimizado']:>15,.0f} "
          f"{row['atendimento_pct']:>9.1f}% {row['lucro_otimizado']:>15,.2f}")

# Calcular métricas finais
print("\n" + "=" * 80)
print("RESULTADOS FINAIS")
print("=" * 80)

# Margem real (baseada em volume projetado limitado por capacidade)
# Como estamos trabalhando por TIPO dentro de GRUPO, o volume_real é a demanda do tipo
# limitada pela capacidade compartilhada do grupo
df_result['volume_real'] = df_result['volume_projetado'].copy()

# Ajustar volume_real por grupo considerando capacidade compartilhada
# Se a demanda total do grupo > capacidade, limitar proporcionalmente por tipo
for grupo in df_result['grupo_capacidade'].dropna().unique():
    df_grupo = df_result[df_result['grupo_capacidade'] == grupo].copy()
    if df_grupo.empty:
        continue
    
    cap_max = df_grupo['capacidade_max'].iloc[0] if df_grupo['capacidade_max'].notna().any() else float('inf')
    volume_real_grupo = df_grupo['volume_real'].sum()
    
    # Se volume_real > capacidade, limitar proporcionalmente mantendo proporção entre tipos
    if volume_real_grupo > cap_max and cap_max < float('inf'):
        fator_limitacao = cap_max / volume_real_grupo
        for idx in df_grupo.index:
            df_result.at[idx, 'volume_real'] = df_result.at[idx, 'volume_real'] * fator_limitacao

# Calcular margens
margem_real = (df_result['base_margem_variavel_unit'] * df_result['volume_real']).sum()
margem_otimizada = df_result['lucro_otimizado'].sum()
melhoria = margem_otimizada - margem_real
melhoria_pct = (melhoria / margem_real * 100) if margem_real > 0 else 0

print(f"\nMargem Real: R$ {margem_real:,.2f}")
print(f"Margem Otimizada: R$ {margem_otimizada:,.2f}")
print(f"Melhoria: R$ {melhoria:,.2f} ({melhoria_pct:.2f}%)")

print(f"\nVolume Real Total: {df_result['volume_real'].sum():,.0f} UC")
print(f"Volume Otimizado Total: {df_result['volume_otimizado'].sum():,.0f} UC")
print(f"Volume Projetado Total: {df_result['volume_projetado'].sum():,.0f} UC")

print(f"\nStatus de Capacidade:")
status_counts = df_result['status_capacidade'].value_counts()
for status, count in status_counts.items():
    print(f"  {status}: {count} SKUs")


VOLUME OTIMIZADO TOTAL POR TIPO

Total de TIPOs: 15
Volume otimizado total: 58,890,804 UC
Lucro otimizado total: R$ 35,847,054.79

TIPO                           GRUPO_CAPACIDADE                  DEMANDA       OTIMIZADO    ATEND.%           LUCRO
--------------------------------------------------------------------------------------------------------------
Pet|2.0                        Pet|2-3L                       49,738,817      25,563,146      51.4%   10,474,559.62
Lata|0.35                      Lata|350ml                     18,122,998      10,668,886      58.9%    9,098,238.99
Pet|0.6                        Pet|600ml                       7,530,622       4,094,682      54.4%    5,082,343.24
Pet|1.5                        Pet|1-1.5L                      5,999,084       3,340,657      55.7%    2,040,912.74
Pet|0.2                        Pet|200ml                       7,732,485       3,324,111      43.0%    1,781,976.88
Pet|1.0                        Pet|1-1.5L                    

In [130]:
# Visualizar resultados por grupo de capacidade
import matplotlib.pyplot as plt

# Resumo por grupo
resumo_grupos = []
for grupo, info in resultados.items():
    resumo_grupos.append({
        'Grupo': grupo,
        'N_TIPOs': info['n_tipos'],
        'Demanda_Total': info['demanda_total'],
        'Volume_Otimizado': info['volume_total'],
        'Lucro_Total': info['lucro_total'],
        'Atendimento_%': info['atendimento_pct'],
        'Status': info['status']
    })

df_resumo = pd.DataFrame(resumo_grupos)
df_resumo = df_resumo.sort_values('Lucro_Total', ascending=False)

print("\n" + "=" * 80)
print("RESUMO POR GRUPO DE CAPACIDADE")
print("=" * 80)
print(df_resumo.to_string(index=False))


RESUMO POR GRUPO DE CAPACIDADE
                     Grupo  N_TIPOs  Demanda_Total  Volume_Otimizado  Lucro_Total  Atendimento_% Status
                  Pet|2-3L        2   4.989062e+07      2.556315e+07 1.047456e+07      51.238381     OK
                Lata|350ml        2   1.863516e+07      1.066889e+07 9.098239e+06      57.251371     OK
                 Pet|600ml        1   7.530622e+06      4.094682e+06 5.082343e+06      54.373757     OK
                Pet|1-1.5L        2   8.972604e+06      6.314176e+06 4.466709e+06      70.371727     OK
                 Pet|200ml        1   7.732485e+06      3.324111e+06 1.781977e+06      42.988917     OK
              KS|290-310ml        1   2.135721e+06      1.926217e+06 1.655255e+06      90.190468     OK
           Mini Lata|220ml        1   2.322536e+06      1.500922e+06 1.142744e+06      64.624268     OK
                 BIB|5-18L        3   2.711957e+06      2.711957e+06 1.060406e+06     100.000000     OK
                     LS|1L      

In [131]:
# Visualizar top SKUs por lucro otimizado
df_result['variacao_volume'] = df_result['volume_otimizado'] - df_result['volume_real']
df_result['variacao_pct'] = (df_result['variacao_volume'] / df_result['volume_real'] * 100).fillna(0)

print("\n" + "=" * 80)
print("TOP 20 SKUs POR LUCRO OTIMIZADO")
print("=" * 80)
top_tipos = df_result.nlargest(20, 'lucro_otimizado')[
    ['tipo', 'grupo_capacidade', 'volume_real', 'volume_otimizado', 
     'variacao_pct', 'lucro_otimizado', 'status_capacidade']
]
print(top_tipos.to_string(index=False))


TOP 20 SKUs POR LUCRO OTIMIZADO
                     tipo           grupo_capacidade  volume_real  volume_otimizado  variacao_pct  lucro_otimizado status_capacidade
                  Pet|2.0                   Pet|2-3L 2.548536e+07      2.556315e+07  3.052004e-01     1.047456e+07                OK
                Lata|0.35                 Lata|350ml 1.037567e+07      1.066889e+07  2.826046e+00     9.098239e+06                OK
                  Pet|0.6                  Pet|600ml 4.094682e+06      4.094682e+06  0.000000e+00     5.082343e+06                OK
                  Pet|1.0                 Pet|1-1.5L 2.092517e+06      2.973519e+06  4.210252e+01     2.425796e+06                OK
                  Pet|1.5                 Pet|1-1.5L 4.221659e+06      3.340657e+06 -2.086863e+01     2.040913e+06                OK
                  Pet|0.2                  Pet|200ml 3.324111e+06      3.324111e+06  0.000000e+00     1.781977e+06                OK
Vidro não Retornável|0.29           

In [132]:
# Salvar resultados
df_result.to_csv('resultado_otimizacao_mix.csv', index=False, decimal=',', encoding='utf-8')
print("\n✓ Resultados salvos em: resultado_otimizacao_mix.csv")


✓ Resultados salvos em: resultado_otimizacao_mix.csv
